# Tarea Computacional: Aplicaciones de Newton-Raphson
---


# Parte de modelación

### Librerías básicas Numpy, Matplotlib, Scipy.linalg, mediapy (agregue más librerías si requiere)

In [2]:
# Other imports and helper functions
from IPython.display import clear_output
from typing import Callable, Optional, Union, List
# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media

''' AGREGA TUS LIBRERIAS '''
import matplotlib.pyplot as plt
import scipy.linalg
import numpy as np

#limpia el mucho texto
clear_output()

---
## P1) Modelamiento de prensa para molde de inyección






OA=36mm, AB=12mm, BC=12mm, GC=30mm, BC⊥GC, CD=37.5mm, e=0, h=60mm

---
## P2) Modelamiento de manipulador robótico SCARA

---
## P3) Modelamiento de bípedo

---
# Parte de simulación (GPU).
Requiere cambio de entorno a GPU. Dado que el uso de GPU se limita por tiempo, no lo active hasta que sea necesario para visualizar.

### Librerías necesarias para la simulación en MuJoCo.
Recuerda cambiar el entorno de ejecución a GPU para la simulación.

In [ ]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')




#Clonacion del repositorio para los modelos
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/uwulises/ME3250_tarea_computacional

clear_output()


## Modelo simulado: SCARA

### Reemplaza los arreglos de joint por tu solución de Newton-Raphson
La pata cuenta con 3 joints, a la cuál se le entrega n arreglos de largo 3. Ojo que el largo de los arreglos debe ser el mismo!

` data_NR := array list((n, 3)) `

In [ ]:
hombro_joint = np.linspace(0,3.14,100)
codo_joint = np.linspace(-2.96,2.96,100)
z_joint = np.linspace(0,0.2,100)

In [ ]:
data_NR_scara = np.array([hombro_joint,codo_joint,z_joint]).T

In [ ]:
%cd
%cd /content/ME3250_tarea_computacional/mujoco/model/scara
with open('scene.xml', 'r') as f:
  xml = f.read()
model_scara = mujoco.MjModel.from_xml_string(xml)
data_scara = mujoco.MjData(model_scara)
renderer = mujoco.Renderer(model_scara,600,800)

/root
/content/ME3250_tarea_computacional/mujoco/model/scara


### Animación del modelo SCARA

In [ ]:
DURATION  = 10   # aprox seconds
FRAMERATE = 60  # Hz
camera = mujoco.MjvCamera()
cam1= mujoco.mjv_defaultFreeCamera(model_scara, camera)
camera.distance = 1.3
camera.azimuth = -90
camera.elevation = 0
camera.orthographic=1

mujoco.mj_resetDataKeyframe(model_scara, data_scara, 1)

frames = []
while data_scara.time < DURATION:
  for i in range(len(data_NR_scara)):
    # Set control vector.
    data_scara.ctrl = data_NR_scara[i]
    # Step the simulation.
    mujoco.mj_step(model_scara, data_scara,nstep=10)
    # Render and save frames.
    if len(frames) < data_scara.time * FRAMERATE:

      camera.lookat = np.array([-0.05, 0.0, 0.3])
      renderer.update_scene(data_scara, camera)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)


This browser does not support the video tag.


## Modelo simulado: Teropobot

### Reemplaza los arreglos de joint por tu solución de Newton-Raphson
La pata cuenta con 5 joints, a la cuál se le entrega n arreglos de largo 5. La lista left_hip_roll_joint se mantiene en 0. Ojo que el largo de los arreglos debe ser el mismo!

` data_NR := array list((n, 5)) `

In [ ]:
left_hip_roll_joint = np.linspace(0,0,100)
left_hip_pitch_joint = np.linspace(0,0.5,100)
left_knee_joint = np.linspace(0.2,0.3,100)
left_ankle_joint = np.linspace(0,0.6,100)
left_metatarsophalangeal_joint = np.linspace(-0.5,0.5,100)

In [ ]:
data_NR_teropobot= np.array([left_hip_roll_joint,left_hip_pitch_joint,left_knee_joint,left_ankle_joint,left_metatarsophalangeal_joint]).T

In [ ]:

%cd
%cd /content/ME3250_tarea_computacional/mujoco/model/teropobot


with open('scene.xml', 'r') as f2:
  xml_teropobot = f2.read()

model_teropobot = mujoco.MjModel.from_xml_string(xml_teropobot)
data_teropobot = mujoco.MjData(model_teropobot)

renderer_2 = mujoco.Renderer(model_teropobot,600,800)


/root
/content/ME3250_tarea_computacional/mujoco/model/teropobot


### Animación del modelo

In [ ]:
DURATION  = 10   # aprox seconds
FRAMERATE = 60  # Hz

# Make a new camera, move it to a closer distance.
camera_2 = mujoco.MjvCamera()

cam2= mujoco.mjv_defaultFreeCamera(model_teropobot, camera_2)
camera_2.distance = 1.3 #og 0.7
camera_2.azimuth = -90
camera_2.elevation = 0
camera_2.orthographic=1
mujoco.mj_resetDataKeyframe(model_teropobot, data_teropobot, 0)
frames_2 = []

while data_teropobot.time < DURATION:
  for i in range(len(data_NR_teropobot)):
    # Set control vector.
    data_teropobot.ctrl = data_NR_teropobot[i]
    # Step the simulation.
    mujoco.mj_step(model_teropobot, data_teropobot,nstep=10)
    # Render and save frames.
    if len(frames_2) < data_teropobot.time * FRAMERATE:

      camera_2.lookat = np.array([-0.05, 0.0, 0.3])
      renderer_2.update_scene(data_teropobot, camera_2)
      pixels = renderer_2.render()
      frames_2.append(pixels)

media.show_video(frames_2, fps=FRAMERATE)

This browser does not support the video tag.
